In [3]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [4]:
data=pd.read_csv("Churn_Modelling.csv")

In [5]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
##PReprocess the data 
##Drop the irrelevan columns 
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [7]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [8]:
##Encode categorical variables 
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [9]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [10]:
##One hot Encoder 'Geographical'ArithmeticError
from sklearn.preprocessing import OneHotEncoder
one=OneHotEncoder()

In [11]:
geoencoder=one.fit_transform(data[['Geography']])

In [12]:
geoencoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [13]:
one.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [14]:
geo_enoceded_df=pd.DataFrame(geoencoder.toarray(),columns=one.get_feature_names_out(['Geography']))


In [15]:
geo_enoceded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [16]:
##Combine one hot encoded columns with the original data 
data=pd.concat([data.drop('Geography',axis=1),geo_enoceded_df],axis=1)

In [17]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [18]:
## Save the encoders and sacler 
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)


with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(one,file)



In [19]:
##Divide the dataset into independent and dependent features 
X=data.drop('Exited',axis=1)
y=data['Exited']

##Split the data into training and testing sets 
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [20]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [21]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Implementation

In [2]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [26]:
##Build our ann model 
model=Sequential()
model.add(Dense(64,activation='relu',input_shape=(X_train.shape[1],)))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))


In [27]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 64)                832       
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)

In [30]:
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [35]:
log_dir='logs/fit'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)
early=EarlyStopping(patience=5,monitor='val_loss',restore_best_weights=True)

In [37]:
history=model.fit(X_train,y_train,batch_size=64,epochs=500,validation_data=(X_test,y_test),callbacks=[tensorflow_callback,early])

Epoch 1/500
125/125 [==============================] - 1s 4ms/step - loss: 0.3319 - accuracy: 0.8608 - val_loss: 0.3395 - val_accuracy: 0.8650
Epoch 2/500
125/125 [==============================] - 0s 4ms/step - loss: 0.3299 - accuracy: 0.8660 - val_loss: 0.3536 - val_accuracy: 0.8610
Epoch 3/500
125/125 [==============================] - 1s 4ms/step - loss: 0.3268 - accuracy: 0.8658 - val_loss: 0.3349 - val_accuracy: 0.8615
Epoch 4/500
125/125 [==============================] - 1s 4ms/step - loss: 0.3251 - accuracy: 0.8636 - val_loss: 0.3482 - val_accuracy: 0.8590
Epoch 5/500
125/125 [==============================] - 1s 4ms/step - loss: 0.3235 - accuracy: 0.8666 - val_loss: 0.3447 - val_accuracy: 0.8605
Epoch 6/500
125/125 [==============================] - 0s 4ms/step - loss: 0.3229 - accuracy: 0.8639 - val_loss: 0.3408 - val_accuracy: 0.8645
Epoch 7/500
125/125 [==============================] - 1s 4ms/step - loss: 0.3206 - accuracy: 0.8654 - val_loss: 0.3516 - val_accuracy: 0.8625

In [38]:
model.save('model.h5')

d:\Churn_Prediction\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [39]:
%load_ext tensorboard

In [44]:
# %tensorboard --logdir logs/fit20240920-001516

In [46]:
##Load the pickle file 
from tensorflow.keras.models import load_model

In [47]:
model=load_model('model.h5')

In [48]:
model

In [49]:
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

In [82]:
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)
with open('onehot_encoder_geo.pkl','rb') as file:
    one=pickle.load(file)

In [51]:
label_encoder_gender

LabelEncoder()

In [52]:
scaler

StandardScaler()

In [93]:
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [94]:
df=pd.DataFrame(input_data,index=[0])

In [95]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [96]:
on=one.transform(df[['Geography']])

In [97]:
on.toarray()

array([[1., 0., 0.]])

In [98]:
geo_enoceded_df=pd.DataFrame(on.toarray(),columns=one.get_feature_names_out(['Geography']))


In [99]:
geo_enoceded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [100]:
input_data=pd.concat([df.reset_index(drop=True),geo_enoceded_df],axis=1)

In [104]:
input_data.drop(['Geography'],axis=1,inplace=True)

In [102]:
input_data['Gender']=label_encoder_gender.transform(input_data['Gender'])

In [110]:
input_data
input_scaled_df=scaler.transform(input_data)


In [113]:
prediction=model.predict(input_scaled_df)[0][0]

1/1 [==============================] - 0s 30ms/step


In [114]:
if prediction>0.5:
    print("The customer is likely to churn ")
else:
    print("The customer is not likely to churn ")

The customer is not likely to churn 
